# Mission to Mars Scrapping

Site to explore: <a href="https://redplanetscience.com/">Redplanet Science</a>

In [2]:
# Importing Dependencies

import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests
from urllib.request import urlopen as uReq
from pprint import pprint

In [89]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Jose.Vazquez\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache




# 1. Latest Mars news extraction

In [90]:
# Defining URL to scrape

url = "https://redplanetscience.com/"
browser.visit(url)

In [91]:
html = browser.html
soup = bs(html, 'html.parser')

title_list = []
art_list = []
lat_new_dict = {}

title = soup.find_all('div', class_='content_title')
article = soup.find_all('div', class_='article_teaser_body')

list_e =[title, article]
i=0

for titl, art in zip(title, article):
    i+=1
    print(f'Title: {titl.text}')
    print(f'Article extract: {art.text}')
    print('----------------------------------')
    title_list.append(titl.text)
    art_list.append(art.text)
    lat_new_dict.update({'title '+str(i):titl.text, 'art '+str(i):art.text})
    
print(lat_new_dict)

Title: 7 Things to Know About the Mars 2020 Perseverance Rover Mission
Article extract: NASA's next rover to the Red Planet is slated to launch no earlier than July 30. These highlights will get you up to speed on the ambitious mission.
----------------------------------
Title: Two Rovers to Roll on Mars Again: Curiosity and Mars 2020
Article extract: They look like twins. But under the hood, the rover currently exploring the Red Planet and the one launching there this summer have distinct science tools and roles to play.
----------------------------------
Title: The Extraordinary Sample-Gathering System of NASA's Perseverance Mars Rover
Article extract: Two astronauts collected Moon rocks on Apollo 11. It will take three robotic systems working together to gather up the first Mars rock samples for return to Earth.
----------------------------------
Title: Naming a NASA Mars Rover Can Change Your Life
Article extract: Want to name the robotic scientist NASA is sending to Mars in 2020? 

In [31]:
dict_l= {'lat_news': {'title':title_list, 'article':art_list}}
pprint(dict_l)

{'lat_news': {'article': ['With the Mars 2020 mission ramping up, the '
                          'documentary — the first of four about past JPL '
                          'missions to the Red Planet to be shown at Caltech — '
                          'tells a gripping backstory.',
                          'New evidence suggests salty, shallow ponds once '
                          "dotted a Martian crater — a sign of the planet's "
                          'drying climate.',
                          'Team members searched for signs of ancient '
                          "microscopic life there, just as NASA's latest rover "
                          'will on the Red Planet next year.',
                          'The small craft will seek to prove that powered, '
                          'controlled flight is possible on another planet. '
                          'But just getting it onto the surface of Mars will '
                          'take a whole lot of ingenuity.',
   

# 2. Featured Image URL build

In [32]:
space_img_url = "https://spaceimages-mars.com/"
browser.visit(space_img_url)

In [33]:
html_img = browser.html
soup_img = bs(html_img, 'html.parser')

header_img = soup_img.find('img', class_="headerimage")['src']

featured_image_url = space_img_url+header_img

print(featured_image_url)


https://spaceimages-mars.com/image/featured/mars2.jpg


# 3. Mars facts table extraction

In [43]:
facts_url = "https://galaxyfacts-mars.com/"
browser.visit(facts_url)

In [73]:
data_tables = pd.read_html(facts_url)
display(data_tables[0])
table_list = data_tables[0].set_index(0).drop('Mars - Earth Comparison')
final_table = table_list.rename(columns={1:'Mars', 2:'Earth' })\
                .to_dict('index')

display(final_table)



,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


{'Diameter:': {'Mars': '6,779 km', 'Earth': '12,742 km'},
 'Mass:': {'Mars': '6.39 × 10^23 kg', 'Earth': '5.97 × 10^24 kg'},
 'Moons:': {'Mars': '2', 'Earth': '1'},
 'Distance from Sun:': {'Mars': '227,943,824 km', 'Earth': '149,598,262 km'},
 'Length of Year:': {'Mars': '687 Earth days', 'Earth': '365.24 days'},
 'Temperature:': {'Mars': '-87 to -5 °C', 'Earth': '-88 to 58°C'}}

# 4. Mars Images extraction

In [92]:
mult_img_url = ("https://marshemispheres.com/")
browser.visit(mult_img_url)

In [93]:
mult_img_html = browser.html
soup = bs(mult_img_html, 'html.parser')

art_list_img = soup.find_all('a', class_='itemLink product-item')
h3_list = soup.find_all('h3')
mars_img_list = {}


for n, h3 in enumerate(h3_list):

    try:                   
        browser.links.find_by_partial_text(h3.text).click()
        crrnt_page = browser.html
        #print(crrnt_page)
        page_soup = bs(crrnt_page, 'html.parser')
        img_download = page_soup.find_all('div', class_='downloads')
        img_href = img_download[0].a['href']
        print(h3.text)
        mars_img_list.update({'title '+str(n):h3.text, 'img url '+str(n):mult_img_url+img_href})
        print('------------------')
        browser.visit(mult_img_url)

    except:
        browser.visit(mult_img_url)
        print("Scraping finished going back to first page") 
        break

print(mars_img_list)


Cerberus Hemisphere Enhanced
------------------
Schiaparelli Hemisphere Enhanced
------------------
Syrtis Major Hemisphere Enhanced
------------------
Valles Marineris Hemisphere Enhanced
------------------
Scraping finished going back to first page
{'title 0': 'Cerberus Hemisphere Enhanced', 'img url 0': 'https://marshemispheres.com/images/full.jpg', 'title 1': 'Schiaparelli Hemisphere Enhanced', 'img url 1': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg', 'title 2': 'Syrtis Major Hemisphere Enhanced', 'img url 2': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg', 'title 3': 'Valles Marineris Hemisphere Enhanced', 'img url 3': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}


In [94]:
    mars_tot_data ={

        'lat_news': {'title':title_list, 'article':art_list},
        'feat_image':featured_image_url,
        'mars_tables': data_tables,
        'hem_list': mars_img_list
    }

In [95]:
pprint(mars_tot_data['hem_list'])

{'img url 0': 'https://marshemispheres.com/images/full.jpg',
 'img url 1': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
 'img url 2': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
 'img url 3': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
 'title 0': 'Cerberus Hemisphere Enhanced',
 'title 1': 'Schiaparelli Hemisphere Enhanced',
 'title 2': 'Syrtis Major Hemisphere Enhanced',
 'title 3': 'Valles Marineris Hemisphere Enhanced'}


In [77]:
pd.DataFrame.from_dict(final_table, orient='index').to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'